# ตัวอย่าง 01: แชทด่วนผ่าน OpenAI SDK

โน้ตบุ๊กนี้แสดงวิธีการใช้ OpenAI SDK ร่วมกับ Microsoft Foundry Local สำหรับการโต้ตอบแบบแชทด่วน

## ภาพรวม

ตัวอย่างนี้แสดง:
- การใช้ OpenAI Python SDK ร่วมกับ Foundry Local
- การจัดการทั้งการตั้งค่า Azure OpenAI และ Foundry Local
- การจัดการข้อผิดพลาดอย่างเหมาะสมและกลยุทธ์สำรอง
- การใช้ FoundryLocalManager สำหรับการจัดการบริการ


## ข้อกำหนดเบื้องต้น

ตรวจสอบให้แน่ใจว่าคุณมี:
1. ติดตั้งและเปิดใช้งาน Foundry Local
2. โหลดโมเดลแล้ว (เช่น `foundry model run phi-4-mini`)
3. ติดตั้งแพ็กเกจ Python ที่จำเป็นเรียบร้อยแล้ว

### ติดตั้ง Dependencies


In [ ]:
# Install required packages if not already installed
!pip install openai foundry-local-sdk

## นำเข้าห้องสมุดที่จำเป็น


In [ ]:
import os
import sys
from openai import OpenAI

try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
    print("✅ Foundry Local SDK is available")
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False
    print("⚠️ Foundry Local SDK not available, will use manual configuration")

## การตั้งค่า

ตั้งค่าการกำหนดค่าของคุณ คุณสามารถเลือกใช้:
1. **Azure OpenAI** (บนคลาวด์)
2. **Foundry Local** (การประมวลผลในเครื่อง)

### ตัวเลือกที่ 1: การตั้งค่า Azure OpenAI


In [ ]:
# Azure OpenAI Configuration (uncomment and set your values)
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource.openai.azure.com"
# os.environ["AZURE_OPENAI_API_KEY"] = "your-api-key"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
# os.environ["MODEL"] = "your-deployment-name"

### ตัวเลือกที่ 2: การกำหนดค่าท้องถิ่นของ Foundry


In [ ]:
# Foundry Local Configuration
MODEL_ALIAS = "phi-4-mini"  # Change this to your preferred model
BASE_URL = "http://localhost:8000"  # Default Foundry Local URL
API_KEY = ""  # Usually not needed for local

## การเริ่มต้นใช้งาน Client

สร้าง OpenAI client ตามการตั้งค่าของคุณ:


In [ ]:
def create_client():
    """Create and configure OpenAI client."""
    
    # Check for Azure OpenAI configuration
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    azure_api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
    
    if azure_endpoint and azure_api_key:
        # Azure OpenAI path
        model = os.environ.get("MODEL", "your-deployment-name")
        client = OpenAI(
            base_url=f"{azure_endpoint}/openai",
            api_key=azure_api_key,
            default_query={"api-version": azure_api_version},
        )
        print(f"🌐 Using Azure OpenAI with model: {model}")
        return client, model
    
    # Foundry Local path
    if FOUNDRY_SDK_AVAILABLE:
        try:
            # Use FoundryLocalManager for proper service management
            manager = FoundryLocalManager(MODEL_ALIAS)
            model_info = manager.get_model_info(MODEL_ALIAS)
            
            # Configure OpenAI client to use local Foundry service
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key  # API key is not required for local usage
            )
            model = model_info.id
            print(f"🏠 Using Foundry Local SDK with model: {model}")
            return client, model
        except Exception as e:
            print(f"⚠️ Could not use Foundry SDK ({e}), falling back to manual configuration")
    
    # Fallback to manual configuration
    client = OpenAI(
        base_url=f"{BASE_URL}/v1",
        api_key=API_KEY
    )
    model = MODEL_ALIAS
    print(f"🔧 Using manual configuration with model: {model}")
    return client, model

# Initialize the client
client, model = create_client()

## ตัวอย่างการแชทพื้นฐาน

มาลองทดสอบการโต้ตอบแชทแบบง่ายๆ กัน:


In [ ]:
def chat_with_model(prompt, max_tokens=128):
    """Send a chat message to the model and get a response."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens
        )
        
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Test with a simple prompt
prompt = "Say hello from Foundry Local and explain what you are in one sentence."
print(f"👤 User: {prompt}")
print("\n🤖 Assistant:")
response = chat_with_model(prompt)
print(response)

## เซสชันแชทแบบโต้ตอบ

ลองใช้คำถามหรือคำสั่งหลากหลายรูปแบบเพื่อดูว่าระบบตอบสนองอย่างไร:


In [ ]:
# Example prompts to try
example_prompts = [
    "What are the benefits of edge AI?",
    "Explain the difference between local and cloud AI inference.",
    "Write a short Python function to calculate the factorial of a number.",
    "What is Microsoft Foundry Local?"
]

for i, prompt in enumerate(example_prompts, 1):
    print(f"\n{'='*50}")
    print(f"Example {i}: {prompt}")
    print(f"{'='*50}")
    
    response = chat_with_model(prompt, max_tokens=200)
    print(response)

## การใช้งานขั้นสูง: การตอบกลับแบบสตรีม

สำหรับการตอบกลับที่ยาวขึ้น การใช้สตรีมสามารถมอบประสบการณ์การใช้งานที่ดีกว่า:


In [ ]:
def chat_with_streaming(prompt, max_tokens=300):
    """Send a chat message with streaming response."""
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            stream=True
        )
        
        print("🤖 Assistant (streaming):")
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        print("\n")  # New line after streaming
        return full_response
    except Exception as e:
        return f"Error: {e}"

# Test streaming with a longer prompt
streaming_prompt = "Explain the advantages of running AI models locally on device versus in the cloud. Include privacy, latency, and cost considerations."
print(f"👤 User: {streaming_prompt}\n")
streaming_response = chat_with_streaming(streaming_prompt)

## การจัดการข้อผิดพลาดและการวินิจฉัย

มาลองตรวจสอบสถานะของบริการและโมเดลที่มีอยู่กัน:


In [ ]:
def check_service_health():
    """Check Foundry Local service health."""
    try:
        # Try to list available models
        models_response = client.models.list()
        models = [model.id for model in models_response.data]
        
        print("✅ Service is healthy")
        print(f"📋 Available models: {models}")
        print(f"🎯 Current model: {model}")
        
        if model in models:
            print("✅ Current model is available")
        else:
            print("⚠️ Current model may not be loaded")
            
    except Exception as e:
        print(f"❌ Service check failed: {e}")
        print("\n🔧 Troubleshooting tips:")
        print("1. Make sure Foundry Local is running")
        print("2. Check if a model is loaded: foundry service ps")
        print("3. Verify the endpoint URL is correct")

check_service_health()

## การทดสอบคำสั่งที่กำหนดเอง

ใช้เซลล์ด้านล่างเพื่อทดสอบคำสั่งของคุณเอง:


In [ ]:
# Enter your custom prompt here
custom_prompt = "Write a haiku about artificial intelligence running on edge devices."

print(f"👤 User: {custom_prompt}\n")
custom_response = chat_with_model(custom_prompt, max_tokens=100)
print(f"🤖 Assistant: {custom_response}")

## สรุป

สมุดบันทึกนี้แสดงให้เห็น:

1. **✅ การตั้งค่าลูกค้า**: วิธีการตั้งค่า OpenAI SDK กับ Foundry Local  
2. **✅ การแชทพื้นฐาน**: การโต้ตอบแบบคำขอ-คำตอบอย่างง่าย  
3. **✅ การสตรีม**: การสร้างคำตอบแบบเรียลไทม์  
4. **✅ การจัดการข้อผิดพลาด**: การจัดการข้อผิดพลาดและการวินิจฉัยที่มีประสิทธิภาพ  
5. **✅ สุขภาพของบริการ**: การตรวจสอบความพร้อมใช้งานของโมเดลและสถานะของบริการ  

### ขั้นตอนถัดไป

- **ตัวอย่าง 02**: การผสานรวม SDK ขั้นสูงพร้อมการสนับสนุน Azure OpenAI  
- **ตัวอย่าง 04**: การสร้างแอปพลิเคชันแชทด้วย Chainlit  
- **ตัวอย่าง 05**: ระบบการจัดการหลายตัวแทน  
- **ตัวอย่าง 06**: การกำหนดเส้นทางโมเดลอัจฉริยะ  

### ประโยชน์สำคัญของ Foundry Local

- 🔒 **ความเป็นส่วนตัว**: ข้อมูลไม่ออกจากอุปกรณ์ของคุณ  
- ⚡ **ความเร็ว**: การคาดการณ์แบบโลคอลที่มีความหน่วงต่ำ  
- 💰 **ต้นทุน**: ไม่มีค่าใช้จ่ายในการใช้งาน API  
- 🔌 **ออฟไลน์**: ทำงานได้โดยไม่ต้องเชื่อมต่ออินเทอร์เน็ต  
- 🛠️ **ความเข้ากันได้**: API ที่เข้ากันได้กับ OpenAI  
